In [0]:
import lsst.daf.persistence as dafPersist
import numpy as np
import os
import lsst.utils
import pfs.drp.stella.findAndTraceAperturesTask as fataTask
import pfs.drp.stella as drpStella
import lsst.afw.display as afwDisplay
display = afwDisplay.Display(1)

In [0]:
testDataDir = os.path.join(lsst.utils.getPackageDir("drp_stella_data"),'tests/data/PFS')
butler = dafPersist.Butler(testDataDir)

In [0]:
dataId = dict(visit=5, ccd=5)
exposure = butler.get('postISRCCD', dataId)

In [0]:
display.mtv(exposure, title="Flat")
display.setMaskTransparency(50)

In [0]:
myFindTask = fataTask.FindAndTraceAperturesTask()
fts = myFindTask.run(exposure)

In [0]:
exposure.getMaskedImage().getMask().addMaskPlane("FIBERTRACE")

ftMask = exposure.getMaskedImage().getMask().getPlaneBitMask("FIBERTRACE")
ftMask = 1 << exposure.getMaskedImage().getMask().getMaskPlane("FIBERTRACE")

for ft in fts.getTraces():
    drpStella.markFiberTraceInMask(ft, exposure.getMaskedImage().getMask(), ftMask)

display.setMaskPlaneColor("FIBERTRACE", "ORANGE")

msk = exposure.getMaskedImage().getMask()
msk.getMaskPlane("FIBERTRACE")
msk &= ~0x2

display.mtv(exposure, title="parent")

In [0]:
with display.Buffering():
    for iTrace in range(fts.size()):
        ft = fts.getFiberTrace(iTrace)
        xCenters = ft.getXCenters()
        ftFunction = ft.getFiberTraceFunction()
        yLow = ftFunction.yCenter + ftFunction.yLow
        yHigh = ftFunction.yCenter + ftFunction.yHigh

        for y in np.arange(yLow, yHigh):
            pointA = [xCenters[y-yLow]-0.5,y]
            pointB = [xCenters[y-yLow+1]-0.5,y+1]
            points = [pointA, pointB]
            display.line(points, ctype='red')